<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Llama_3_2_3B_SFT_GGML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Fine-Tuning Meta-Llama-3.2-3B Used unsloth for CPU and GPU Inference - GGML**

On September 25, 2024, Meta introduced Llama 3.2, a collection of multilingual large language models (LLMs) in 1B and 3B sizes. These models are optimized for multilingual dialogue use cases, including agentic retrieval and summarization tasks. Notably, the Llama 3.2 1B and 3B models support a context length of 128K tokens, making them suitable for extensive text processing tasks.
HUGGING FACE

To access the Llama 3.2-1B model, you can download it from [Hugging Face](https://huggingface.co/unsloth/Llama-3.2-3B-Instruct) The approval process is typically swift, often taking about 20 minutes.
HUGGING FACE

### Table of Contents
1. Install dependancies
2. Download model
3. Fintuning flow
4. convert GGML formate


## Step 1: Install All the Required Packages

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## Step 2: Import necessary libraries Load model and tokenizer

In [ ]:
# Import necessary libraries
from unsloth import FastLanguageModel
import torch

# Configuration settings
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",  # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",  # Use token if using gated models like meta-llama/Llama-2-7b-hf
)


### We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

### **Load the dataset**

In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# Assuming tokenizer is already defined, and chat_template is set to "llama-3.1"
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# Function to format the prompts based on conversation examples
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

# Load the dataset and slice to get the first 500 records
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
dataset_500 = dataset.select(range(500))  # Select only the first 500 records

# Apply formatting function to the 500-record subset
formatted_dataset = dataset_500.map(formatting_prompts_func, batched=True)
